In [1]:
from PIL import Image
from utils import seed_everything
from image_patch import ImagePatch, llm_query, best_image_match, distance, bool_to_yesno

seed_everything()

import ast
from vision_processes import forward
from torchvision import transforms
import requests

2024-11-30 05:53:48.778631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 05:53:49.458502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-11-30 05:53:49.458566: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-11-30 05:53:49.458572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/deepkapha/anaconda3/envs/vipergpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /home/deepkapha/anaconda3/envs/vipergpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using cache found in /home/deepkapha/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /home/deepkapha/.cache/torch/hub/intel-isl_MiDaS_master


VISION BACKBONE USE GRADIENT CHECKPOINTING:  False
LANGUAGE BACKBONE USE GRADIENT CHECKPOINTING:  False
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B
EARLY FUSION ON, USING MHA-B


In [2]:
def get_code(query):
    model_name_codex = 'hominiscode'
    code = forward(model_name_codex, prompt=query, input_type="image")
    code = ast.unparse(ast.parse(code))
    return code

def load_image(path):
    if path.startswith("http://") or path.startswith("https://"):
        image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
        image = transforms.ToTensor()(image)
    else:
        image = Image.open(path)
        image = transforms.ToTensor()(image)
    return image

In [3]:
im = load_image('test/car.jpeg')
query = 'What would the founder of the brand of the car on the left say to the founder of the brand of the car on the right?'
input_type = "image"

In [7]:
import time  # Importing time to introduce delay between retries, if needed

MAX_RETRIES = 3  # Number of retries allowed
retry_count = 0

while retry_count < MAX_RETRIES:
    try:
        # Original code logic
        code = get_code(query)
        code = code.replace('execute_command(image, my_fig, time_wait_between_lines, syntax)', 'execute_command(image)')
        exec(compile(code, 'hominiscode', 'exec'), globals())
        result = globals()['execute_command'](im)
        break  # Exit loop if execution is successful
    except AttributeError as e:
        retry_count += 1
        print(f"Attempt {retry_count} failed: {e}")
        if retry_count == MAX_RETRIES:
            code = None
            result = "Cannot run analysis for this query image pair"
        else:
            time.sleep(1)  # Optional delay between retries

In [8]:
print(code)

def execute_command(image):
    car_patches = ImagePatch(image).find('car')
    car_patch_left = car_patches[0]
    car_patch_right = car_patches[1]
    brand_left = car_patch_left.simple_query('What is the name of the brand of this car?')
    brand_right = car_patch_right.simple_query('What is the name of the brand of this car?')
    left_quote = llm_query(f'What would the founder of {brand_left} say to the founder of {brand_right}?', long_answer=True)
    right_quote = llm_query(f'What would the founder of {brand_right} say to the founder of {brand_left}?', long_answer=True)
    return f"The founder of {brand_left} would say: '{left_quote}' and the founder of {brand_right} would say: '{right_quote}'"


In [9]:
result

'The founder of ferrari would say: \'What a fascinating question!\n\nLet\'s imagine the conversation between Enzo Ferrari, the founder of Ferrari, and Ferruccio Lamborghini, the founder of Lamborghini.\n\nEnzo Ferrari: "Ferruccio, my friend, I\'ve heard you\'re trying to take on me and my beloved Ferrari. Tell me, what drives your passion for building cars?"\n\nFerruccio Lamborghini: "Ah, Enzo, it\'s about innovation, quality, and beating the status quo! You see, as a tractor manufacturer and a lover of fast cars, I knew I had to challenge your dominance in the industry."\n\nEnzo Ferrari: "Ha! The audacity! But seriously, Ferruccio, what makes you think you can take on the likes of me? I\'ve spent decades perfecting the art of building exquisite machines that embody speed, style, and sophistication."\n\nFerruccio Lamborghini: "Ah, but Enzo, that\'s exactly where we differ. You may have perfected the art of building cars for the elite, but I\'m committed to creating vehicles for the peo